# Data pre-processing. Keister Zooplankton Hood Canal 2012-13 data

University of Washington Pelagic Hypoxia Hood Canal project, Zooplankton dataset.

10/30, 4/5,4, 3/27-25,20 2023

## Responses from Julie's team

> Calyptopis 1-3 life_history_stage codes. For Euphasia Pacifica, calyptopis life stages are typically coded in life_history_stage as "Cal1;_Calyptopis_1", "Cal2;_Calyptopis_2" and "Cal3;_Calyptopis_3" (same for thysanoessa). But there are a few Euphasia records that include the following codes: "1;_CI", "2;_CII", "3;_CIII". These are the same life_history_stage codes used for copepods, copepodites C1 - C3. My guess is that they're miscoded and should be calyptopis 1-3. Can you confirm?

Yes, these are miscoded ("1, CI" should be "Cal1, calyptopis 1," and so on).

> "F1_0;_Furcilia_1_0_legs" life_history_stage code. Is this a Furcilia 10 life stage?

No, this is a Furcilia 1 life stage. I believe this just means there is 1 set of legs with no setae. Amanda can confirm this.



Here are answers to the rest of your questions:

> Time zone. The dataset includes "date", "time_start" and "time" columns, where I believe the latter was generated by BCO-DMO from the other two columns provided by Julie. The parameter description says that time (actually date + time) is in UTC, but based on the correspondence to the "day_nigth" column and the N/D code in the sample code, I think that's wrong and it's actually in PDT. Could you confirm? 

I spot checked some of the samples, and they are in local time, so that would have been PST/PDT. 
I checked the data sheets, and those times that BethElLee sent are correct.

> "F1_0;_Furcilia_1_0_legs" life_history_stage code. Is this a Furcilia 10 life stage?

I checked this data sheet, and it looks like there was a data entry error. I found the example you were looking at in sample 20130613DBNm1_335. This life history stage should be, "F10;_Furcilia_10."


> I have another question that may be just curiosity but might help me indirectly. The sample code in the dataset is packed with information and looks like 20130906DBDm1_200, which based on the BCO-DMO description is: "PI issued sample ID; sampling date + Station + D (day) or N (night) + Net code (e.g. m1) _mesh". However, there are 50 unique sample codes that have 1-3 extra characters between the Station and the D/N codes. Most of those cases have a single character, like A, B or i; a smaller number has the characters "ii" or "iii". Could you explain what those extra characters represent? My guess based on inspecting the data is that those cases involved a need to do additional net tows at the station.

That's a good question! We had another boat out there with us checking for interesting acoustic signatures, such as suspected krill aggregations. Some of our zooplankton tows were called "acoustic" tows, meaning that they were done in an area where the other boat had just found something interesting- We called that an "acoustic target station." So we had "acoustic" and "non-acoustic" tows. For non-acoustic tows, we just used the station name, such as "Union." If we did replicate tows for this, they would be: Union i, Union ii, Union, iii, etc. for a 200 um tow, and Union I, Union II, etc. for 335 um. The acoustic target station here would have been Union A, and replicate tows would have been Union B, Union C, etc. (always 335 um).

## Other notes, from DwC Event notebook

Parse the data to define and pull out 3 event "types": `cruise`, `stationVisit` and `sample`. The DwC event table will be populated sequentially for each of those event types, in that order, from the most temporally aggregated (cruise) to the most granular (sample). Columns will be populated differently depending on the event type. Since a cruise is a spatial collection of points, a `footprintWKT` polygon and depth ranges are generated and populated.

**Notes about fields to use**
- mention why I'm not using `id` columns (see Abby's comments)
- consider adding `basisOfRecord`
- See additional fields used in the LifeWatch dataset: modified, language, rightsHolder, accessRights, institutionCode, datasetName, country (in addition to id, type, eventID, parentEventID, samplingProtocol, eventDate, locationID, waterBody, countryCode, minimumDepthInMeters, maximumDepthInMeters, decimalLatitude, decimalLongitude)

**date-time issues**
- Are times in UTC (as claimed by the `time` variable) or PDT?? Based on `day_night`, I think they're actually in PDT, not UTC!
- Inconsistencies between `time_start` and `day_night`. I've spotted at least one: sample ("20120611UNDm3_200") labelled as "Day" while `time_start` is "23:10". Clearly one of them is wrong. Test for other obvious inconsistencies by setting a time window for Day vs Night and comparing it to `time_start`.
- Missing `time_start` values. There are a few such cases, but apparently no corresponding missing `date` values. Still, they lead to `NaT` `time` values. I need to replace the `NaT`'s with a valid datetime, and the only option is to use the `date` and an artificial time value. Start with a fixed value (12:00), then refine it by using 12:00 when `day_night` is `Day` and 23:00 when it's `Night`.
- Times capture only the *start* of the sampling event. So, using times as the event end in an interval range would be misleading.

**Other comments**
- Add "Day sampling" / "Night sampling" to `eventRemarks` for sample events (or stationVisit, or both?), based on `day_night` column
- Parsing `sample_code` for distinctive information
  - Example `sample_code`: "20120611UNDm2_200". Dataset description entry for `sample_code`: "PI issued sample ID; sampling date + Station + D (day) or N (night) + Net code (e.g. m1) \_mesh"
  - The upper case letter character before the "m" is D or N (Day or Night). **In a few cases there's an additional character found before the D/N character, but its meaning is not described in the `sample_code` description**
- These were monthly cruises lasting about 4 days, in June-October 2012 and 2013 (ie, 8 cruises).
- DONE. Look up US `countryCode`

In [1]:
from datetime import datetime, timedelta, timezone
import json
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
data_pth = Path(".")

## Pre-process data from csv for Event table

### Read the csv file

In [3]:
sourcecsvdata_pth = data_pth / "sourcedata" / "bcodmo_dataset_682074_data.csv"

`usecols` defines the columns that will be kept and the order in which they'll be organized

In [4]:
source_df = pd.read_csv(sourcecsvdata_pth, skiprows=[1])

In [5]:
len(source_df)

6884

In [6]:
source_df.head()

,station,latitude,longitude,day_night,species,date,time_start,life_history_stage,sample_code,mesh_size,depth_max,depth_min,FWC_DS,density,time
0,DB,47.378,-123.117,Day,ACARTIA,20131003,14:11,3;_CIII,20131003DBDm2_200,200,56.0,30.0,DS,6.395349,2013-10-03T14:11:00Z
1,DB,47.378,-123.117,Day,ACARTIA,20130906,16:17,5;_CV,20130906DBiDm1_200,200,60.0,0.0,FWC,38.759690,2013-09-06T16:17:00Z
2,DB,47.378,-123.117,Day,ACARTIA,20131003,NaN,Female;_Adult,20131003DBDm1_200,200,56.0,0.0,FWC,0.384615,NaN
3,DB,47.378,-123.117,Day,ACARTIA,20131003,NaN,Male;_Adult,20131003DBDm1_200,200,56.0,0.0,FWC,1.538462,NaN
4,DB,47.378,-123.117,Day,ACARTIA_CLAUSI,20120614,17:25,Female;_Adult,20120614DBDm3_200,200,20.0,9.0,DS,21.052632,2012-06-14T17:25:00Z


Some `time` entries are missing (NaT). It looks like it's because the time component is missing, while the date component is available.

In [7]:
len(source_df[source_df['time'].isnull()])

168

## Fix time issues

- Assign correct timezone (PDT, not UTC)
- Fill in missing `time_start`, by `sample_code`

In [8]:
missing_times = {
    "20131003DBDm1_200": "14:11",
    "20130904HPNm1_200": "21:59",
    "20130903UNDm1_335": "15:01",
    "20121001UNDm1_200": "14:12",
    "20120709UNNm5_335": "22:10",
    "20120709UNNm4_335": "22:45"
}

In [9]:
source_df.loc[source_df.sample_code == "20130903UNDm1_335"].head(3)

,station,latitude,longitude,day_night,species,date,time_start,life_history_stage,sample_code,mesh_size,depth_max,depth_min,FWC_DS,density,time
4086,UN,47.812,-122.807,Day,CHAETOGNATHA,20130903,NaN,Unknown,20130903UNDm1_335,335,68.0,40.0,DS,0.642857,NaN
4118,UN,47.812,-122.807,Day,CLIONE_LIMACINA,20130903,NaN,Unknown,20130903UNDm1_335,335,68.0,40.0,DS,0.357143,NaN
4196,UN,47.812,-122.807,Day,CYPHOCARIS_CHALLENGERI,20130903,NaN,Unknown,20130903UNDm1_335,335,68.0,40.0,DS,1.071429,NaN


Set the `time_start` strings for the `sample_code` entries in `missing_times`.

In [10]:
for sample_code,time_start in missing_times.items():
    source_df.loc[source_df["sample_code"] == sample_code, "time_start"] = time_start

Create (replace) the `time` column based on `date`, `time_start`, my custom `pdt` timezone, and `strftime`

In [11]:
pdt = timezone(timedelta(hours=-8), "PDT")

source_df["time"] = pd.to_datetime(
    source_df["date"].astype(str) + source_df["time_start"], 
    format="%Y%m%d%H:%M"
).dt.tz_localize(pdt)

In [12]:
source_df.loc[source_df.sample_code == "20130903UNDm1_335"].head(3)

,station,latitude,longitude,day_night,species,date,time_start,life_history_stage,sample_code,mesh_size,depth_max,depth_min,FWC_DS,density,time
4086,UN,47.812,-122.807,Day,CHAETOGNATHA,20130903,15:01,Unknown,20130903UNDm1_335,335,68.0,40.0,DS,0.642857,2013-09-03 15:01:00-08:00
4118,UN,47.812,-122.807,Day,CLIONE_LIMACINA,20130903,15:01,Unknown,20130903UNDm1_335,335,68.0,40.0,DS,0.357143,2013-09-03 15:01:00-08:00
4196,UN,47.812,-122.807,Day,CYPHOCARIS_CHALLENGERI,20130903,15:01,Unknown,20130903UNDm1_335,335,68.0,40.0,DS,1.071429,2013-09-03 15:01:00-08:00


## Replace incorrect `life_history_stage` values

- Simple replacement of "F1_0;_Furcilia_1_0_legs" entry with "F10;_Furcilia_10". Use pandas `replace` on the column
- EUPHAUSIA_PACIFICA and THYSANOESSA in `species` column: replace `life_history_stage` based on combined `species` and `life_history_stage` entries
  > Calyptopis 1-3 life_history_stage codes. For Euphasia Pacifica, calyptopis life stages are typically coded in life_history_stage as "Cal1;_Calyptopis_1", "Cal2;_Calyptopis_2" and "Cal3;_Calyptopis_3" (same for thysanoessa). But there are a few Euphasia records that include the following codes: "1;_CI", "2;_CII", "3;_CIII". These are the same life_history_stage codes used for copepods, copepodites C1 - C3. My guess is that they're miscoded and should be calyptopis 1-3. Can you confirm?

  Yes, these are miscoded ("1, CI" should be "Cal1, calyptopis 1," and so on).

In [13]:
source_df["life_history_stage"].replace({"F1_0;_Furcilia_1_0_legs": "F10;_Furcilia_10"}, inplace=True)

In [14]:
sel_species = source_df["species"].isin(["EUPHAUSIA_PACIFICA", "THYSANOESSA"])

krill_bad_life_history_stages = {
    "1;_CI": "Cal1;_Calyptopis_1", 
    "2;_CII": "Cal2;_Calyptopis_2",
    "3;_CIII": "Cal3;_Calyptopis_3",
}

In [15]:
for old_lhs,new_lhs in krill_bad_life_history_stages.items():
    source_df.loc[sel_species & source_df["life_history_stage"].str.startswith(old_lhs), "life_history_stage"] = new_lhs

### Parse `life_history_stage`

#### Parse into `lhs_0` and `lhs_1` tokens

In [16]:
source_df[['lhs_0', 'lhs_1']] = pd.DataFrame(
    source_df['life_history_stage'].str.split(';_').to_list(), 
    index=source_df.index
)

In [17]:
source_df[['life_history_stage', 'lhs_0', 'lhs_1']].value_counts(dropna=False, sort=False).sort_index(ascending=True)

life_history_stage  lhs_0             lhs_1       
1;_CI               1                 CI               120
2;_CII              2                 CII              176
3;_CIII             3                 CIII             272
4;_CIV              4                 CIV              492
5;_CV               5                 CV               611
Adult               Adult             NaN              194
Bract               Bract             NaN                6
Cal1;_Calyptopis_1  Cal1              Calyptopis_1      49
Cal2;_Calyptopis_2  Cal2              Calyptopis_2      34
Cal3;_Calyptopis_3  Cal3              Calyptopis_3      42
Copepodite          Copepodite        NaN              265
Cyphonaut           Cyphonaut         NaN               81
Cyprid_larva        Cyprid_larva      NaN               36
Egg                 Egg               NaN               91
Eudoxid             Eudoxid           NaN                5
F10;_Furcilia_10    F10               Furcilia_10        3
F1;_F

### Examine original issues with incorrect entries

**NOTE:** The statements below will show the original issues only when run with the `lhs_0` and `lhs_1` columns parsed from the *original, unaltered* `life_history_stage` entries. I've commented them out for that reason.

Examine use of CI-V life stages (Copepodites?) with EUPHAUSIA_PACIFICA krill. Note that only CI-III are used with krill, and only a small subset of the totals (9, 9 & 6, respectively)

In [18]:
# source_df[source_df.species == 'EUPHAUSIA_PACIFICA'][['lhs_0', 'lhs_1']].value_counts(dropna=False).sort_index(ascending=True)

Confirm that 'Cal1', 'Cal2', 'Cal3' are only used with krill

In [19]:
# pd.DataFrame(
#     source_df[source_df.lhs_0.isin(['Cal1', 'Cal2', 'Cal3'])]
#     [['lhs_0', 'lhs_1', 'species']]
#     .value_counts(dropna=False)
#     .sort_index(ascending=True)
# ).head(60)

In [20]:
# pd.DataFrame(
#     source_df.lhs_0.isin(['1', '2', '3', '4', '5'])
#     [['lhs_0', 'lhs_1', 'species']]
#     .value_counts(dropna=False)
#     .sort_index(ascending=True)
# ).head(60)

## Parse `sample_code`

### Examine `sample_code` characteristics, extra characters

In [21]:
source_df.sample_code.str.len().value_counts()

17    5837
18     824
19     185
20      34
14       4
Name: sample_code, dtype: int64

In [22]:
source_df[source_df.sample_code.str.len().isin([14,19,20])].head()

,station,latitude,longitude,day_night,species,date,time_start,life_history_stage,sample_code,mesh_size,depth_max,depth_min,FWC_DS,density,time,lhs_0,lhs_1
2664,DU,47.404,-123.125,Day,AETIDEUS,20120808,18:49,3;_CIII,20120808DUiiDm1_200,200,165.0,0.0,FWC,4.624277,2012-08-08 18:49:00-08:00,3,CIII
2665,DU,47.404,-123.125,Day,AETIDEUS,20120808,18:49,5;_CV,20120808DUiiDm1_200,200,165.0,0.0,FWC,2.312139,2012-08-08 18:49:00-08:00,5,CV
2666,DU,47.404,-123.125,Day,AETIDEUS,20120808,18:49,Female;_Adult,20120808DUiiDm1_200,200,165.0,0.0,FWC,6.936416,2012-08-08 18:49:00-08:00,Female,Adult
2671,DU,47.404,-123.125,Day,AGLANTHA,20120808,18:49,Unknown,20120808DUiiDm1_200,200,165.0,0.0,FWC,4.624277,2012-08-08 18:49:00-08:00,Unknown,None
2676,DU,47.404,-123.125,Day,BIVALVIA,20120808,18:49,Veliger,20120808DUiiDm1_200,200,165.0,0.0,FWC,99.421965,2012-08-08 18:49:00-08:00,Veliger,None


In [23]:
source_df[source_df.sample_code.str.len() == 18].head()

,station,latitude,longitude,day_night,species,date,time_start,life_history_stage,sample_code,mesh_size,depth_max,depth_min,FWC_DS,density,time,lhs_0,lhs_1
1,DB,47.378,-123.117,Day,ACARTIA,20130906,16:17,5;_CV,20130906DBiDm1_200,200,60.0,0.0,FWC,38.759690,2013-09-06 16:17:00-08:00,5,CV
28,DB,47.378,-123.117,Day,AETIDEUS,20130906,16:17,3;_CIII,20130906DBiDm1_200,200,60.0,0.0,FWC,3.875969,2013-09-06 16:17:00-08:00,3,CIII
29,DB,47.378,-123.117,Day,AETIDEUS,20130906,16:17,5;_CV,20130906DBiDm1_200,200,60.0,0.0,FWC,3.875969,2013-09-06 16:17:00-08:00,5,CV
92,DB,47.378,-123.117,Day,BIVALVIA,20130906,16:17,Veliger,20130906DBiDm1_200,200,60.0,0.0,FWC,100.775194,2013-09-06 16:17:00-08:00,Veliger,None
114,DB,47.378,-123.117,Day,BRYOZOAN,20130906,16:17,Cyphonaut,20130906DBiDm1_200,200,60.0,0.0,FWC,27.131783,2013-09-06 16:17:00-08:00,Cyphonaut,None


### Extract net_code and "extra token" from `sample_code`

- Retain only ones that are not already found among the existing dataframe columns.
- Example: "20120611UNDm2_200". Dataset description entry for `sample_code`: "PI issued sample ID; sampling date + Station + D (day) or N (night) + Net code (e.g. m1) \_mesh"
- The upper case letter character before the "m" is D or N (Day or Night). **In a few cases there's an additional character found before the D/N character, but its meaning is not described in the `sample_code` description**
- Ultimately, try to pull out or create a profile code and profile depth interval code, if appropriate?

In [24]:
len(source_df.sample_code.unique())

271

Parsing steps:
- split the new `sample_code` on the "_" delimiter, create two new columns, `token1` and `mesh_size`
- From `token1` extract `token2`, the characters between `station_code` and the "_" delimiter
- parse `token2`: split on the letter "m", into `token3` and `net_code`; then create `token4` from `token3` by removing the D/N character. `token4` will be empty in most cases, and will be renamed to `extra_sample_token`

In [25]:
def split_token2(token2):
    token3, net_code = token2.split('m')
    token4 = token3[:-1]
    
    return pd.Series({
        'net_code': 'm'+net_code,
        'extra_sample_token': token4
    })

source_df['token2'] = source_df['sample_code'].apply(
    lambda cd: cd[10:].split('_')[0]
)
source_df = pd.concat([source_df, source_df['token2'].apply(split_token2)], 
                            axis='columns')
source_df.drop(columns='token2', inplace=True)

In [26]:
source_df.head(10)

,station,latitude,longitude,day_night,species,date,time_start,life_history_stage,sample_code,mesh_size,depth_max,depth_min,FWC_DS,density,time,lhs_0,lhs_1,net_code,extra_sample_token
0,DB,47.378,-123.117,Day,ACARTIA,20131003,14:11,3;_CIII,20131003DBDm2_200,200,56.0,30.0,DS,6.395349,2013-10-03 14:11:00-08:00,3,CIII,m2,
1,DB,47.378,-123.117,Day,ACARTIA,20130906,16:17,5;_CV,20130906DBiDm1_200,200,60.0,0.0,FWC,38.759690,2013-09-06 16:17:00-08:00,5,CV,m1,i
2,DB,47.378,-123.117,Day,ACARTIA,20131003,14:11,Female;_Adult,20131003DBDm1_200,200,56.0,0.0,FWC,0.384615,2013-10-03 14:11:00-08:00,Female,Adult,m1,
3,DB,47.378,-123.117,Day,ACARTIA,20131003,14:11,Male;_Adult,20131003DBDm1_200,200,56.0,0.0,FWC,1.538462,2013-10-03 14:11:00-08:00,Male,Adult,m1,
4,DB,47.378,-123.117,Day,ACARTIA_CLAUSI,20120614,17:25,Female;_Adult,20120614DBDm3_200,200,20.0,9.0,DS,21.052632,2012-06-14 17:25:00-08:00,Female,Adult,m3,
5,DB,47.378,-123.117,Day,ACARTIA_CLAUSI,20120614,17:25,Male;_Adult,20120614DBDm3_200,200,20.0,9.0,DS,21.052632,2012-06-14 17:25:00-08:00,Male,Adult,m3,
6,DB,47.378,-123.117,Day,ACARTIA_CLAUSI,20120614,17:25,Female;_Adult,20120614DBDm4_200,200,9.0,0.0,DS,70.833333,2012-06-14 17:25:00-08:00,Female,Adult,m4,
7,DB,47.378,-123.117,Day,ACARTIA_CLAUSI,20120712,16:23,5;_CV,20120712DBDm1_200,200,70.0,45.0,DS,5.128205,2012-07-12 16:23:00-08:00,5,CV,m1,
8,DB,47.378,-123.117,Day,ACARTIA_CLAUSI,20121004,15:00,Female;_Adult,20121004DBDm4_200,200,30.0,15.0,DS,5.217391,2012-10-04 15:00:00-08:00,Female,Adult,m4,
9,DB,47.378,-123.117,Day,ACARTIA_CLAUSI,20130808,15:54,5;_CV,20130808DBDm4_200,200,10.0,0.0,DS,280.851064,2013-08-08 15:54:00-08:00,5,CV,m4,


In [27]:
len(source_df)

6884

## Handle "duplicate" records

Records that are identical except for density measurements. Calculate the average of the duplicates, then "drop" duplicates (ie, keep a single record for each duplicate). See `duplicate-records-except-density`

Unique combinations of these three columns correspond to what should be a unique record.

In [28]:
cols = ['sample_code', 'species', 'life_history_stage']

Identify the records that are duplicated, as the unique combination of `cols` columns. This is `duplicated_df`. `all_dups_source_df` then stores all the records matching these duplicated records (x2) but now adding the `density` column.

In [29]:
duplicated_df = source_df[source_df[cols].duplicated()][cols]
all_dups_source_df = source_df[cols + ['density']].merge(duplicated_df, on=cols, how="inner")

Calculate mean density for each pair of duplicated records.

In [30]:
mean_density_df = all_dups_source_df.groupby(cols).mean().reset_index().rename(columns={"density":"density_avg"})

Merge the average-density dataframe with `source_df`. The net effect is to add a new column, `density_avg`

In [31]:
source_meandensity_df = source_df.merge(mean_density_df, on=cols, how="left")

The `sel` approach below is based on matching multi-indices in the two dataframes, using the 3-column `cols` set. (from [here](https://stackoverflow.com/questions/54006298/select-rows-of-a-dataframe-based-on-another-dataframe-in-python))

For the 34 duplicated records only, assign `density_avg` to the `density` column. Then clean up: drop the `density_avg` column and, finally, drop what are now true duplicates (ie, 17 records). 

`source_final_df` is the final, de-duplicated, density-averaged dataframe.

In [32]:
sel = source_meandensity_df.set_index(cols).index.isin(mean_density_df.set_index(cols).index)
source_meandensity_df.loc[sel, 'density'] = source_meandensity_df[sel]['density_avg']

source_final_df = source_meandensity_df.drop(columns=["density_avg"]).drop_duplicates()

In [33]:
len(duplicated_df), len(all_dups_source_df), len(source_df), len(source_df) - len(source_final_df), len(source_final_df)

(17, 34, 6884, 17, 6867)

## Move the taxonomy replacements here??

No, b/c the taxonomy notebook also does a lot of other taxonomy querying and mappings, so it's best to keep it all there.

## Package versions

In [34]:
print(
    f"{datetime.utcnow()} +00:00\n"
    f"pandas: {pd.__version__}, geopandas: {gpd.__version__}"
)

2023-11-01 02:47:27.815453 +00:00
pandas: 1.5.3, geopandas: 0.12.2
